***I. IMPORT AND CLEAN DATASETS***

**IMPORT LIBRARIES**

In [2]:
import pandas as pd
import numpy as np

**LOAD AND INSPECT THE DATASETS**

First, we need to look at the results data stored in results.csv.

In [3]:
results = pd.read_csv('datasets/results.csv')
print(results.info())
results.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47399 entries, 0 to 47398
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        47399 non-null  object
 1   home_team   47399 non-null  object
 2   away_team   47399 non-null  object
 3   home_score  47399 non-null  int64 
 4   away_score  47399 non-null  int64 
 5   tournament  47399 non-null  object
 6   city        47399 non-null  object
 7   country     47399 non-null  object
 8   neutral     47399 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 2.9+ MB
None


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


Overview:
- There are 47399 football games played globally in total.
- Currently, there are no missing data in any column.
- The column names are correct.

Let's load and inspect the goalscorers dataset.

In [4]:
goalscorers = pd.read_csv('datasets/goalscorers.csv')
print(goalscorers.info())
goalscorers.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44362 entries, 0 to 44361
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       44362 non-null  object 
 1   home_team  44362 non-null  object 
 2   away_team  44362 non-null  object 
 3   team       44362 non-null  object 
 4   scorer     44313 non-null  object 
 5   minute     44103 non-null  float64
 6   own_goal   44362 non-null  bool   
 7   penalty    44362 non-null  bool   
dtypes: bool(2), float64(1), object(5)
memory usage: 2.1+ MB
None


,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False


Some observations:
- This dataset isn't quite clear, there are 44362 'goalscorers' in total, but there exists players who score a double, a hattrick, etc.
- However, there aren't missing values in any column.

For further analysis, we have another dataset, also specifying football games data. Let's have a look.

In [5]:
shootouts = pd.read_csv('datasets/shootouts.csv')
shootouts.info()
shootouts.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644 entries, 0 to 643
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           644 non-null    object
 1   home_team      644 non-null    object
 2   away_team      644 non-null    object
 3   winner         644 non-null    object
 4   first_shooter  230 non-null    object
dtypes: object(5)
memory usage: 25.3+ KB


,date,home_team,away_team,winner,first_shooter
0,1967-08-22,India,Taiwan,Taiwan,NaN
1,1971-11-14,South Korea,Vietnam Republic,South Korea,NaN
2,1972-05-07,South Korea,Iraq,Iraq,NaN
3,1972-05-17,Thailand,South Korea,South Korea,NaN
4,1972-05-19,Thailand,Cambodia,Thailand,NaN


For this dataset, some of the matches are far dated, so we should only take matches date from the 1990s and forward.



**CLEANING THE DATASETS**


Let's first cleaning the 2 datasets: goalscorers.csv and results.csv.

In [6]:
matches = pd.merge(
    left = results,
    right = goalscorers,
    left_on = ['date', 'home_team', 'away_team'],
    right_on = ['date', 'home_team', 'away_team'],
    how = 'inner'
)
matches.head()


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,team,scorer,minute,own_goal,penalty
0,1916-07-02,Chile,Uruguay,0,4,Copa América,Buenos Aires,Argentina,True,Uruguay,José Piendibene,44.0,False,False
1,1916-07-02,Chile,Uruguay,0,4,Copa América,Buenos Aires,Argentina,True,Uruguay,Isabelino Gradín,55.0,False,False
2,1916-07-02,Chile,Uruguay,0,4,Copa América,Buenos Aires,Argentina,True,Uruguay,Isabelino Gradín,70.0,False,False
3,1916-07-02,Chile,Uruguay,0,4,Copa América,Buenos Aires,Argentina,True,Uruguay,José Piendibene,75.0,False,False
4,1916-07-06,Argentina,Chile,6,1,Copa América,Buenos Aires,Argentina,False,Argentina,Alberto Ohaco,2.0,False,False


We have a new dataset, including the matches' informations and also the scorers in those matches.

Now, let's also merge the final dataset: goalscorers.csv to see which countries are winners.

In [7]:
matches_winners = pd.merge(
    left = matches,
    right = shootouts,
    left_on = ['date', 'home_team', 'away_team'],
    right_on = ['date', 'home_team', 'away_team'],
    how = 'left'
)

def lastpreview(dataframe): #preview 5 last rows of dataframe
    return dataframe.iloc[-5:]

matches_winners.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44362 entries, 0 to 44361
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           44362 non-null  object 
 1   home_team      44362 non-null  object 
 2   away_team      44362 non-null  object 
 3   home_score     44362 non-null  int64  
 4   away_score     44362 non-null  int64  
 5   tournament     44362 non-null  object 
 6   city           44362 non-null  object 
 7   country        44362 non-null  object 
 8   neutral        44362 non-null  bool   
 9   team           44362 non-null  object 
 10  scorer         44313 non-null  object 
 11  minute         44103 non-null  float64
 12  own_goal       44362 non-null  bool   
 13  penalty        44362 non-null  bool   
 14  winner         357 non-null    object 
 15  first_shooter  314 non-null    object 
dtypes: bool(3), float64(1), int64(2), object(10)
memory usage: 4.5+ MB


***II. VISUALIZING AND ANALYZING***

As we can see, the dataset is now good enough for analyzing, except that it has several null objects - notably in the goalscorers column. We should left this column behind, as it does not affect our work below.

For analysis, we need to answer 3 questions:
- Do neutral stadiums affect scores' rate?
- What are the rate of own goals compared to scored goals in each tournament?
- Do neutral stadiums, in any way, affect the ratio of own goals?

For the first question, we need to first look at the sum of the total of home goals and away goals, played in neutral stadiums (not a stadium of any team in match).

In [8]:
is_neutral = matches_winners['neutral'] == True
sum_of_goals_neutral = np.abs(matches_winners[is_neutral]['home_score'].sum() + matches_winners[is_neutral]['away_score'].sum())
sum_of_goals_neutral

49898

At the same time, we also want to check how many matches are played in a neutral stadium:

In [9]:
neutral_match = matches_winners[is_neutral]['date'].count()
neutral_match

11571

We can now calculate the ratio of goals per match played in a neutral bowl.

In [10]:
rt_neutral = (sum_of_goals_neutral / neutral_match)
rt_neutral

4.312332555526748

Compare to matches played in a non-neutral stadium:

In [11]:
sum_of_goals_non_neutral = np.abs(matches_winners[~is_neutral]['home_score'].sum() + matches_winners[~is_neutral]['away_score'].sum())
normal_match = matches_winners[~is_neutral]['date'].count()
rt_non_neutral = sum_of_goals_non_neutral / normal_match
rt_non_neutral

4.166722576316673

As we can compare above, matches that are played in a third stadium is seemed to rise the number of goals scored, with a rate of 4.3 goals per match, while matches in normal stadiums the rate slightly decreases to 4.1 goals per match. 

Moving on to the second question, we first have to group the database by tournaments.

In [20]:
results = pd.read_csv('datasets/results.csv')
extracted_column = goalscorers['own_goal']
results.insert(9, 'own_goal', extracted_column)
results['total_goals'] = results['home_score'] + results['away_score']
comparison = results.groupby('tournament').agg({'total_goals':'mean', 'own_goal':'mean'})
comparison.head(n=10)


,total_goals,own_goal
tournament,,
ABCS Tournament,3.700000,0.0
AFC Asian Cup,2.657957,0.02973
AFC Asian Cup qualification,3.319372,0.012431
AFC Challenge Cup,2.660000,0.03
AFC Challenge Cup qualification,3.043478,0.021739
AFF Championship,3.422680,0.008368
AFF Championship qualification,3.516129,0.016667
Africa Cup of Nations qualification,2.666667,0.016667
African Cup of Nations,2.378310,0.020319


It is clear that most of the tournaments have the rate of own goals lower than those of scored ones. 

Finally, let's add a new column 'neutral_count' to the dataset below and compare the ratio of own goals, by filtering from highest to lowest.

In [19]:
#re-import the dataset
results = pd.read_csv('datasets/results.csv')
extracted_column = goalscorers['own_goal']
results.insert(9, 'own_goal', extracted_column)
results['total_goals'] = results['home_score'] + results['away_score']
comparison = results.groupby('tournament').agg({'total_goals':'mean', 'own_goal':'mean', 'neutral':'mean'})

#adding a new column
comparison.head(n=10).sort_values(by='own_goal', ascending=False)

,total_goals,own_goal,neutral
tournament,,,
AFC Challenge Cup,2.660000,0.03,0.800000
AFC Asian Cup,2.657957,0.02973,0.783848
AFC Challenge Cup qualification,3.043478,0.021739,0.478261
African Cup of Nations,2.378310,0.020319,0.799496
African Cup of Nations qualification,2.424200,0.020263,0.043785
AFF Championship qualification,3.516129,0.016667,0.548387
Africa Cup of Nations qualification,2.666667,0.016667,0.016667
AFC Asian Cup qualification,3.319372,0.012431,0.344241
AFF Championship,3.422680,0.008368,0.446735


It can be marked that except for some tournaments (Bolivarian Games, ...), the ratio of neutral matchs seems not to affect the rate of own goals per match.